In [7]:
# cifar-10,100のインポート

In [6]:
import os
import pickle
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from scipy import ndimage, misc
from sklearn import datasets
from sklearn.model_selection import train_test_split
%matplotlib inline
# 乱数シードを指定
np.random.seed(seed=0)

In [29]:
#CIFAR-10のデータセットのインポート
from keras.datasets import cifar10
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
 
#CIFAR-10の正規化
from keras.utils import to_categorical
 
 
# 特徴量の正規化
X_train = X_train/255.
X_test = X_test/255.
 
# クラスラベルの1-hotベクトル化
Y_train = to_categorical(Y_train, 10)
Y_test = to_categorical(Y_test, 10)
 


In [10]:
# CNNの構築
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
import numpy as np
 
model = Sequential()
 
model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))
 
# コンパイル
model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
 
#訓練
history = model.fit(X_train, Y_train, epochs=20)
 


Epoch 1/20
1563/1563 [==============================] - 207s 132ms/step - loss: 2.1018 - accuracy: 0.2160
Epoch 2/20
1563/1563 [==============================] - 266s 170ms/step - loss: 1.8135 - accuracy: 0.3438
Epoch 3/20
1563/1563 [==============================] - 314s 201ms/step - loss: 1.6153 - accuracy: 0.4142
Epoch 4/20
1563/1563 [==============================] - 32408s 21s/step - loss: 1.4783 - accuracy: 0.4652
Epoch 5/20
1563/1563 [==============================] - 336s 215ms/step - loss: 1.3901 - accuracy: 0.4991
Epoch 6/20
1563/1563 [==============================] - 586s 375ms/step - loss: 1.3133 - accuracy: 0.5266
Epoch 7/20
1563/1563 [==============================] - 180s 115ms/step - loss: 1.2542 - accuracy: 0.5519
Epoch 8/20
1563/1563 [==============================] - 191s 122ms/step - loss: 1.1964 - accuracy: 0.5731
Epoch 9/20
1563/1563 [==============================] - 33052s 21s/step - loss: 1.1421 - accuracy: 0.5938
Epoch 10/20
1563/1563 [=======================

In [16]:
# モデルの保存
model.save('./CIFAR-10.h5')

 #評価 & 評価結果出力
print(model.evaluate(X_test, Y_test))

313/313 [==============================] - 7s 21ms/step - loss: 0.7757 - accuracy: 0.7311
[0.775740921497345, 0.7311000227928162]


In [30]:
from keras.models import load_model
# 保存したモデル構造の読み込み
model=load_model('./CIFAR-10.h5')
#モデルの図示化
#from keras.utils import plot_model
#plot_model(model, to_file='model.png',show_shapes=True)

In [74]:
import numpy as np
# cifar-10 : (60000, 32, 32), (60000)
# cifar-100 : (60000, 32, 32), (60000)
# X_test : (10000, 32, 32)
# Y_test : (10000, 10)
print(X_test.shape)
print(Y_test.shape)
Y_test0 = np.full((10000, 10), 0.1)
score0 = model.evaluate(X_test, Y_test0)

#CIFAR-100のデータセットのインポート
from keras.datasets import cifar100
(X_train2, Y_train2), (X_test2, Y_test2) = cifar100.load_data()
#CIFAR-100の正規化
from keras.utils import to_categorical
# 特徴量の正規化
X_train2 = X_train2/255.
X_test2 = X_test2/255.
 
# クラスラベルの1-hotベクトル化
Y_train2 = to_categorical(Y_train2, 100)
Y_test2 = to_categorical(Y_test2, 100)

score2 = model.evaluate(X_test2, Y_test0)

(10000, 32, 32, 3)
(10000, 10)
313/313 [==============================] - 10s 33ms/step - loss: 5.0375 - accuracy: 0.0818


In [142]:
# cifar-10, cifar-100からそれぞれ100ずつデータを取ってきている。
# x_judge : (200, 3072) : 予測精度確認用の画像の入力データセット
# y_test0 : 200個のデータセットに対して、[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1] を並べたもの。
# y_test0はcifar-10における10クラス分類の予測の確信度が低い場合の極端な例で、これとのlossが少ないものほど予測の確信度が低い。
# cifar-10の中での10クラス分類がうまくいかないと言える。これはcifar-100のデータである可能性が高い。
# したがって、lossの値を適切に選べば精度良くcifar-100を識別する事ができる可能性がある。

y_test01_10 = np.full((200, 10), 0.1)
x_judge = np.zeros((200, 32, 32, 3))
x_judge[0:100,:,:,:] = X_test[0:100,:,:,:]
x_judge[100:,:,:,:] = X_test2[100:200,:,:,:]
# y_judge : (20000,)
y_judge = np.full(200, 0.0)
y_judge[100:] = np.full(100, 1.0)

In [86]:
# 10か100かの予測が実際と一致するものの数をcntとして全体の数200で割っている。
cnt=0.0
for i in range(200) :
    loss = model.evaluate(x_judge[i:i+1,:,:,:], y_test01_10[i:i+1,:], verbose=0)
    j=0
    if loss[0]>4.5 :
        j=0
    else :
        j=1
    if((j+y_judge[i])%2==0):
        cnt+=1.0
ans = cnt / 200
print(ans)

0.735


In [37]:
# 他にも[0, 0, 0, 0, 0, 0.2, 0.2, 0.2, 0.2, 0.2]や[0, 0, 0, 0, 0, 0, 0, 0.3, 0.3, 0.3]およびその並べ替えも同様に確信度が低い場合の例とも言える。
# これらとのlossが1つでも閾値より低くなるものをcifar-100だと判定し、余ったものをcifar-10と判定するという手法も考えられる。

# 質問
# 1. 上の結果に見られるように、いちいち一つ一つのデータに対してloss, accuracyが表示されてしまうのが邪魔なので、
# これを表示せずに済む方法が知りたいです。

# 2. そもそもevaluateで計算したlossで評価するという手段ではなく画像のテストデータ一つ一つ(32, 32, 3)を入力した際に出力される
# 10次元のベクトル一つ一つを個別に出力してくれるような方法があれば、もっと多彩な分類の手法を考える事ができると思うのですが、ございませんでしょうか？

# 3. そもそもこれはプロダクト演習課題として方向性はあっていますでしょうか？

# もし提出後のコメントで書き切れないなどございましたら、スラック(E8_宮澤)の方までよろしくお願いします。

In [48]:
print(model(X_test[0:10]))

tf.Tensor(
[[2.60726199e-03 6.43920386e-03 2.43544243e-02 5.87193191e-01
  9.51061025e-03 3.17169756e-01 2.20778529e-02 5.72229829e-03
  2.17169523e-02 3.20843863e-03]
 [5.04850131e-03 4.78794649e-02 2.20090101e-07 6.64371214e-07
  5.63912295e-07 5.25096988e-09 1.16825227e-07 2.06310666e-08
  9.46238101e-01 8.32341262e-04]
 [2.51927614e-01 1.56580463e-01 6.12395466e-04 1.16430549e-03
  1.50298618e-03 1.88400692e-04 5.00966707e-05 4.23302670e-04
  5.22754431e-01 6.47959486e-02]
 [8.69546294e-01 7.12927058e-02 2.31577782e-03 3.99876444e-04
  2.66440446e-03 1.71652155e-05 5.39367975e-05 1.29121268e-04
  4.54218537e-02 8.15887190e-03]
 [3.01709804e-07 6.67113636e-06 8.78077652e-03 1.57298446e-02
  4.76600796e-01 4.60283831e-04 4.98412251e-01 3.87908631e-06
  3.26485815e-06 1.97755662e-06]
 [3.10242467e-04 4.78375114e-05 1.84456613e-02 5.42542152e-02
  2.87854988e-02 6.01013824e-02 8.35984826e-01 1.13111909e-03
  3.15987243e-04 6.23241183e-04]
 [5.30163897e-03 6.20212913e-01 7.55687768e-04 

In [84]:
print(model(X_test2[0:10]))

tf.Tensor(
[[1.23154737e-01 2.17369852e-05 8.61484051e-01 2.10772525e-03
  3.83140915e-03 2.21965951e-03 1.51762587e-03 3.64096509e-03
  1.90531090e-03 1.16669762e-04]
 [6.24308595e-04 1.31569075e-04 2.36771293e-02 1.86453149e-01
  6.26048088e-01 7.01806992e-02 8.90301913e-02 2.82949558e-03
  2.82205234e-04 7.43188488e-04]
 [2.84713745e-01 2.12535658e-03 9.31821316e-02 1.83536243e-02
  4.04288650e-01 1.08966036e-02 1.01619549e-01 4.90913689e-02
  1.34417573e-02 2.22872011e-02]
 [7.38110067e-03 2.74858743e-01 1.84525661e-02 7.13306442e-02
  3.43018442e-01 1.27640888e-01 6.36382028e-02 1.57760568e-02
  1.71314680e-03 7.61902481e-02]
 [2.46926606e-01 7.70152872e-03 1.62102599e-02 9.20615904e-03
  2.45854408e-02 1.68967224e-03 3.77826695e-03 9.55157855e-04
  6.81717336e-01 7.22956145e-03]
 [1.56546617e-03 1.21596804e-05 1.39751613e-01 1.35554969e-02
  7.72635281e-01 4.88019735e-03 6.68988079e-02 5.98220271e-04
  5.25299984e-05 5.02549992e-05]
 [3.70685803e-03 1.56746944e-04 1.78717777e-01 

In [58]:
# CNNの構築
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
import numpy as np
 
model = Sequential()
 
model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

In [59]:
# コンパイル
model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
 
#訓練
history = model.fit(X_train, Y_train, epochs=20)

Epoch 1/20
1563/1563 [==============================] - 232s 149ms/step - loss: 1.7824 - accuracy: 0.3468
Epoch 2/20
1563/1563 [==============================] - 235s 150ms/step - loss: 1.4040 - accuracy: 0.4885
Epoch 3/20
1563/1563 [==============================] - 365s 233ms/step - loss: 1.2446 - accuracy: 0.5508
Epoch 4/20
1563/1563 [==============================] - 232s 148ms/step - loss: 1.1254 - accuracy: 0.5981
Epoch 5/20
1563/1563 [==============================] - 229s 146ms/step - loss: 1.0460 - accuracy: 0.6263
Epoch 6/20
1563/1563 [==============================] - 228s 146ms/step - loss: 0.9786 - accuracy: 0.6532
Epoch 7/20
1563/1563 [==============================] - 2610s 2s/step - loss: 0.9215 - accuracy: 0.6735
Epoch 8/20
1563/1563 [==============================] - 14396s 9s/step - loss: 0.8727 - accuracy: 0.6922
Epoch 9/20
1563/1563 [==============================] - 965s 617ms/step - loss: 0.8280 - accuracy: 0.7085
Epoch 10/20
1563/1563 [==========================

In [65]:
# モデルの保存
model.save('./CIFAR-10_3.h5')

 #評価 & 評価結果出力
print(model.evaluate(X_test, Y_test))

313/313 [==============================] - 11s 35ms/step - loss: 0.6598 - accuracy: 0.7779
[0.659820020198822, 0.777899980545044]


In [66]:
model(X_test[0:100])

<tf.Tensor: shape=(100, 10), dtype=float32, numpy=
array([[5.62016357e-05, 7.03644910e-05, 7.46814301e-04, 8.78888845e-01,
        7.72072759e-04, 3.41867134e-02, 8.44731554e-02, 4.81655181e-04,
        2.57846201e-04, 6.63054452e-05],
       [1.04973000e-03, 8.27026553e-04, 2.20928155e-06, 3.18763364e-06,
        5.87160912e-07, 3.43846907e-08, 2.56046383e-06, 5.77424437e-08,
        9.98082280e-01, 3.23197164e-05],
       [2.04074918e-03, 1.72093662e-03, 7.09885353e-05, 1.18104035e-04,
        4.25161925e-05, 4.56715679e-06, 1.67752296e-04, 8.41081965e-06,
        9.95449603e-01, 3.76415323e-04],
       [3.57829720e-01, 1.60535844e-03, 1.62651181e-01, 2.59883236e-03,
        7.20475568e-03, 1.34329792e-04, 1.38825766e-04, 1.37085590e-04,
        4.67570424e-01, 1.29431821e-04],
       [1.02793308e-06, 7.35552476e-07, 9.49415285e-03, 2.62614246e-03,
        8.08657799e-03, 4.15166105e-05, 9.79747295e-01, 4.51851008e-07,
        1.27247802e-06, 8.78955177e-07],
       [7.80515256e-06, 

In [83]:
model(X_test2[0:100])

<tf.Tensor: shape=(100, 10), dtype=float32, numpy=
array([[1.23154737e-01, 2.17369652e-05, 8.61484051e-01, 2.10772525e-03,
        3.83141125e-03, 2.21965951e-03, 1.51762587e-03, 3.64096346e-03,
        1.90531090e-03, 1.16669762e-04],
       [6.24308537e-04, 1.31569192e-04, 2.36771274e-02, 1.86453134e-01,
        6.26048028e-01, 7.01807365e-02, 8.90301466e-02, 2.82949535e-03,
        2.82205205e-04, 7.43188430e-04],
       [2.84713805e-01, 2.12535542e-03, 9.31821391e-02, 1.83536224e-02,
        4.04288620e-01, 1.08966026e-02, 1.01619527e-01, 4.90913652e-02,
        1.34417564e-02, 2.22871993e-02],
       [7.38110440e-03, 2.74858773e-01, 1.84525680e-02, 7.13306665e-02,
        3.43018383e-01, 1.27640843e-01, 6.36381879e-02, 1.57760568e-02,
        1.71314576e-03, 7.61902556e-02],
       [2.46926606e-01, 7.70152872e-03, 1.62102599e-02, 9.20615904e-03,
        2.45854408e-02, 1.68967131e-03, 3.77826695e-03, 9.55157855e-04,
        6.81717336e-01, 7.22956145e-03],
       [1.56546617e-03, 

In [67]:
def step_decay(epoch):
    lr = 0.001
    if(epoch >= 100):
        lr/=5
    if(epoch>=140):
        lr/=2
    return lr

In [87]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
# コンパイル
model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])


val_gen = ImageDataGenerator().flow(X_test,Y_test, batch_size)
lr_decay = LearningRateScheduler(step_decay)
history = model.fit_generator(da_generator(), epochs=epochs, steps_per_epoch = steps_per_epoch,\
                        validation_data = val_gen, validation_steps = validation_steps,callbacks = [lr_decay])

my_eval(model,x_test,t_test)

NameError: name 'batch_size' is not defined

In [234]:
loss = model.evaluate(X_test[0:200], y_test01_10)
loss = model.evaluate(X_test2[0:200], y_test01_10)

7/7 [==============================] - 0s 33ms/step - loss: 4.9277 - accuracy: 0.0650


In [239]:
# y_judge : (200,)
y_judge = np.full(200, 0)
y_judge[100:] = np.full(100, 1)
# y_test1_10 : (200, 10)
y_test1_10= np.full((200, 10), 0.0)
# 10か100かの予測が実際と一致するものの数をcntとして全体の数200で割っている。
cnt=0.0
j = np.full(200, 1)


In [249]:
#j = j_save
j = np.zeros(200)
for i in range(200) :
    loss = model.evaluate(x_judge[i:i+1,:,:,:], y_test01_10[i:i+1,:], verbose=0)
    if loss[0]<6:
            j[i] =1

In [245]:
for i_x in range(10):
            y_test1_10[:,:]=0.0
            y_test1_10[:,i_x] = 1.0
            for i in range(200) :
                loss = model.evaluate(x_judge[i:i+1,:,:,:], y_test1_10[i:i+1,:], verbose=0)
                if loss[0]<0.3 :
                    #if j[i] == 1 :
                        j[i] = 0
j_save = j.copy()

In [250]:
j = j_save
cnt_1 = 0.0
cnt_2 = 0.0
for i in range(100) :
    print(j[i])
    print(y_judge[i])
    if(j[i] == y_judge[i]):
        cnt_1+=1.0
for i in range(100) :
    print(j[i+100])
    print(y_judge[i+100])
    if(j[i+100] == y_judge[i+100]):
        cnt_2+=1.0
ans_1 = cnt_1 / 100
ans_2 = cnt_2 / 100
ans = (cnt_1+cnt_2) / 200
print(ans_1)
print(ans_2)
print(ans)

0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
1
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
1
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
0
1
0
1
0
1
1
1
1
1
1
1
1
1
0
1
0
1
0
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
0
1
0
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
0
1
1
1
1
1
0.75
0.71
0.73


In [141]:
# [1.0,0.0, ... , 0.0], loss < 0.5 : cifar-10...0.8, cifar-100... 0.53
# [1.0,0.0, ... , 0.0], loss < 0.7 : cifar-10...0.92, cifar-100... 0.37
# [1.0,0.0, ... , 0.0], loss < 0.8 : cifar-10...0.95, cifar-100... 0.29
# [1.0,0.0, ... , 0.0], loss < 0.9 : cifar-10...0.97, cifar-100... 0.24
# [0.1, 0.1, ... , 0.1], loss < 5.0 : cifar-10...0.75, cifar-100...0.71
# [0.1, 0.1, ... , 0.1], loss < 6.0 : cifar-10...0., cifar-100...0.